In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import hydra
import itertools
import mlflow
import numpy as np
import os
import pandas as pd
import pathlib
import plotly.express as px
import torch
import xarray as xr
import yaml

import omegaconf

In [ ]:
from smc01.postprocessing.lightning import SMC01Module
from smc01.postprocessing.cli.util import find_checkpoint_file, make_datasets, make_dataloader

In [ ]:
from smc01.postprocessing.util import load_checkpoint_from_run

In [ ]:
DATA_DIR = pathlib.Path(os.getenv('DATA_DIR'))

In [ ]:
with hydra.initialize_config_module('smc01.postprocessing.conf'):
    cfg = hydra.compose('validate', ['experiment=attention_gdps_metar', 'experiment.model.big_features_embedding=False'])

In [ ]:
train_dataset, val_dataset, test_dataset, n_stations, n_features = make_datasets(cfg)

In [ ]:
val_dataloader = make_dataloader(cfg, val_dataset, shuffle=False, concat_collate=False)

In [ ]:
CHECKPOINT_DIR = DATA_DIR / 'runs/postprocessing/multirun/2022-05-31/13-53-07/4854681'

In [ ]:
model = hydra.utils.instantiate(cfg.experiment.model, 1226, 18)
module = SMC01Module.load_from_checkpoint(find_checkpoint_file(CHECKPOINT_DIR), model=model, full_stations_file=DATA_DIR / 'interpolated/2021-12-20-gdps-metar/stations.csv')

In [ ]:
one_example = next(iter(val_dataloader))

In [ ]:
pred = model.forward(one_example)

In [ ]:
pred

In [ ]:
model.station_embedding.shape

In [ ]:
affinity = torch.matmul(model.station_embedding, model.station_embedding.T).detach().numpy()

In [ ]:
affinity.shape

In [ ]:
px.imshow(affinity.detach().numpy())

In [ ]:
len(val_dataset.stations)

In [ ]:
val_dataset.stations.index('CYUL')

In [ ]:
affinity[437]

In [ ]:
stations = pd.read_csv(DATA_DIR / 'interpolated/2021-12-20-gdps-metar/stations_w_metadata.csv')

In [ ]:
stations

In [ ]:
to_append = pd.DataFrame([{'station': None, 'latitude': None, 'longitude': None, 'elevation': None}])

In [ ]:
stations = pd.concat([stations, to_append])

In [ ]:
stations

In [ ]:
val_dataset.stations.index('OAK')

In [ ]:
stations['cyul_affinity'] = affinity[437]

In [ ]:
stations['oak_affinity'] = affinity[901]

In [ ]:
stations

In [ ]:
px.scatter_geo(data_frame=stations, lat='latitude', lon='longitude', color='oak_affinity', hover_name='station')